In [10]:
#using Pkg

#Pkg.activate("/Users/tims/.julia/dev/FUSE_Design")
using Revise
using FUSE

using Plots;
FUSE.logging(Logging.Info; actors=Logging.Error);


In [11]:
negative_triangularity = false

ini,act = FUSE.case_parameters(:KDEMO);

### Act settings
act.ActorEquilibrium.model = :TEQUILA
act.ActorCoreTransport.model = :FluxMatcher

act.ActorPFdesign.model=:optimal
act.ActorFluxSwing.operate_oh_at_j_crit = true # this maximizes flattop inside the fluxswing actor

act.ActorWholeFacility.update_plasma = true

# This is handeled by the constraint funcitons
act.ActorHFSsizing.error_on_performance = false
act.ActorHFSsizing.error_on_technology = false


act.ActorCoreTransport.model = :FluxMatcher
act.ActorFluxMatcher.evolve_densities = :flux_match


ini.core_profiles.zeff = ini.core_profiles.zeff = 2.0 ↔ [1.1, 4.0]
ini.core_profiles.ne_setting = :greenwald_fraction_ped
ini.core_profiles.ne_value = 0.5 ↔ [0.2, 1.3]
ini.core_profiles.impurity = :Kr  ↔ (:Kr, :Ar, :Ne, :Xe)

act.ActorFluxMatcher.max_iterations = 500


ini.equilibrium.ζ = 0.1 ↔ [0,0.2]
ini.equilibrium.B0 = ini.equilibrium.B0 ↔ [3.0, 15.0]
ini.equilibrium.ip = ini.equilibrium.ip ↔ [4.0e6, 22e6]
ini.equilibrium.R0 = ini.equilibrium.R0 ↔ [4.0, 8.0]
ini.equilibrium.pressure_core = missing

ini.bop.cycle_type = :brayton ↔ (:rankine, :brayton)
ini.tf.technology = :rebco #  ↔ (:nb3sn_iter, :rebco)
ini.tf.shape = :racetrack ↔ (:racetrack,:double_ellipse,:offset)
ini.oh.technology = :rebco # ↔ (:nb3sn_iter, :rebco)
ini.pf_active.technology = :nb3sn_iter # ↔ (:nb3sn_iter, :rebco)



if negative_triangularity

    act.ActorPedestal.model = :WPED
    act.ActorWPED.ped_to_core_fraction = 0.25

    act.ActorFluxMatcher.rho_transport = 0.2:0.05:0.9
    act.ActorTGLF.tglfnn_model ="sat0quench_em_d3d+mastu_azf+1"
    act.ActorFluxMatcher.algorithm = :simple
    act.ActorFluxMatcher.evolve_pedestal = false

    ini.equilibrium.δ = -0.4 ↔ [-0.7, 0.0]

    ini.equilibrium.κ =  1.8
    ini.tf.shape = :racetrack ↔ (:racetrack,:double_ellipse,:offset)

    ini.requirements.lh_power_threshold_fraction = missing
    
    
else
    
    act.ActorPedestal.model = :EPED
    act.ActorFluxMatcher.rho_transport = 0.2:0.05:0.8
    act.ActorTGLF.tglfnn_model ="sat0quench_em_d3d+mastu_azf+1"
    act.ActorFluxMatcher.algorithm = :simple
    act.ActorFluxMatcher.evolve_pedestal = true

    ini.equilibrium.δ = 0.4 ↔ [0.0, 0.7]

    ini.equilibrium.κ =  0.9
    ini.tf.shape = :offset ↔ (:double_ellipse,:offset)

    ini.requirements.lh_power_threshold_fraction = 1.

end    

# Requirements
ini.requirements.flattop_duration = 3600.
ini.requirements.log10_flattop_duration = missing # log10 versionis not needed

ini.requirements.power_electric_net = 250e6 # 250 +/- 50 MWe
ini.requirements.tritium_breeding_ratio = 1.1
ini.requirements.q95 = 3.0
ini.requirements.beta_normal = 3.5
ini.requirements.Psol_R = 15. # this pushes you to have a big tokamak

ini.requirements.coil_j_margin = 0.1
ini.requirements.coil_stress_margin = 0.1


IMAS.update_ObjectiveFunctionsLibrary!()
IMAS.update_ConstraintFunctionsLibrary!()

OFL = deepcopy(IMAS.ObjectiveFunctionsLibrary)
CFL = deepcopy(IMAS.ConstraintFunctionsLibrary)

objective_functions = [OFL[:min_capital_cost],OFL[:max_q95]]#, OFL[:max_log10_flattop]]

constraint_functions = []

if negative_triangularity
    constraint_functions = [
        CFL[:power_electric_net],
        CFL[:min_q95],
        CFL[:max_transport_error],
        CFL[:max_beta_normal],
        CFL[:max_Psol_R],
        CFL[:max_tf_coil_j],CFL[:max_oh_coil_j],
        CFL[:max_pl_stress],CFL[:max_tf_coil_stress],CFL[:max_oh_coil_stress]]
   
else
    constraint_functions = [
        CFL[:power_electric_net],
        CFL[:min_q95],
        CFL[:max_transport_error],
        CFL[:max_beta_normal],
        CFL[:min_lh_power_threshold_fraction],
        CFL[:max_Psol_R],
        CFL[:max_tf_coil_j],CFL[:max_oh_coil_j],
        CFL[:max_pl_stress],CFL[:max_tf_coil_stress],CFL[:max_oh_coil_stress]]
end

println("== OBJECTIVE FUNCTIONS ==")
display(objective_functions)
println()
println("== CONSTRAINT FUNCTIONS ==")
display(constraint_functions)

== OBJECTIVE FUNCTIONS ==


2-element Vector{IMAS.ObjectiveFunction}:
 min_capital_cost → -Inf [$B]
 max_q95 → Inf []


== CONSTRAINT FUNCTIONS ==


11-element Vector{IMAS.ConstraintFunction}:
 power_electric_net == 0.0 ± 0.2 [%]
 min_q95 > 0.0 [%]
 max_transport_error < 0.1 []
 max_beta_normal < 0.0 []
 min_lh_power_threshold_fraction > 1.0 [%]
 max_Psol_R < 0.0 [%]
 max_tf_coil_j < 1.0 [%]
 max_oh_coil_j < 1.0 [%]
 max_pl_stress < 1.0 [%]
 max_tf_coil_stress < 1.0 [%]
 max_oh_coil_stress < 1.0 [%]

In [12]:
# sty is the act equivalent for a study, it has common parameters like server and n_workers but also study dependent parameters like n_simulations
sty, _ = FUSE.study_parameters(:MultiObjectiveOptimizer);
typeof(sty)

FUSE.FUSEparameters__ParametersStudyMultiObjectiveOptimizer{Real}

In [13]:
# Interacting with sty
sty.server = "localhost" # this can be set to saga/omega/your_cluster
sty.n_workers = 4

dirr = joinpath(pwd(),"test_moop212123223")
if !isdir(dirr)
    sty.save_folder = mkdir(dirr)
else
    sty.save_folder = dirr
end

sty.restart_workers_after_n_generations = 5 # this is the default behavior and releases workers after running the study

sty.population_size = 4 
sty.number_of_generations = 4

# For a reaslistic study you need about population_size = 300 and number_of_generations = 100

sty

ParametersStudyMultiObjectiveOptimizer
├─ server{String} ➡ "localhost" Where to run ["localhost", "omega", "saga", "feynman", "engaging"]
├─ n_workers{Int64} ➡ 4 Number of workers to run with
├─ file_save_mode{Symbol} ➡ :safe_write Saving file policy, `safe_write` only writes when the folder is empty [:safe_write,
│  :overwrite, :append]
├─ release_workers_after_run{Bool} ➡ true Releases the workers after running the study
├─ restart_workers_after_n_generations{Int64} ➡ 5 Runs the optimization in a safe way restarting the workers every N
│  generations, default is never restart
├─ save_folder{String} ➡ "/Users/tims/Dropbox/FusionPowerPlantDesign_FUSE/FuseExamples/test_moop212123223" Folder to save
│  the database runs into
├─ population_size{Int64} ➡ 4 Number of individuals in a generation
├─ number_of_generations{Int64} ➡ 4 Number generations
└─ database_policy{Symbol} ➡ :single_hdf5 Data storage policy: 'separate_folders' stores each case in a separate folder,
   while 'single_hdf5' 

In [14]:
# study is the actor equivalent of actors, here sty, act and outputs are kept of the workflow
study = FUSE.StudyMultiObjectiveOptimizer(sty,ini, act, constraint_functions, objective_functions); # it is possible to pass in keyword arguments to sty
# instantiating the study will also setup the study i.e. FUSE.setup(study);

# for now you will have to import FUSE everywhere in your distributed computing
using Distributed
@everywhere import FUSE
@everywhere import IJulia

Working with 4 workers on Tims-MacBook-Pro.local


In [15]:
Distributed.workers()

4-element Vector{Int64}:
 2
 3
 4
 5

In [16]:
max_gens_per_iteration = sty.restart_workers_after_n_generations

steps = Int(ceil(sty.number_of_generations / max_gens_per_iteration))
sty.restart_workers_after_n_generations
1 == steps, mod(sty.number_of_generations, max_gens_per_iteration)

(true, 4)

In [ ]:
FUSE.run(study);   # runs the study

# at the end of the run workers are released so you will have to setup again if you want to run more

Running 5 generations (1 / 1)
Working with 4 workers on Tims-MacBook-Pro.local
Running on 4 worker processes
== Actuators ==
ini.equilibrium.B0
Float64ype: 
- units: T
- description: Vacuum toroidal field at R0 [T]; Positive sign means anti-clockwise when viewing from above. The product R0B0 must be consistent with the b_tor_vacuum_r field of the tf IDS.
- value: 6.55
- base: 6.55
- default: missing
- opt: SimulationParameters.OptParameterRange{Float64}(6.55, 3.0, 15.0, 1, missing)
- check: nothing
ini.equilibrium.R0
- type: Float64
- units: m
- description: Geometric genter of the plasma. NOTE: This also scales the radial build layers.
- value: 6.8
- base: 6.8
- default: missing
- opt: SimulationParameters.OptParameterRange{Float64}(6.8, 4.0, 8.0, 1, missing)
- check: #162
ini.equilibrium.δ
- type: Float64
- units: -
- description: Triangularity of the plasma boundary [MXH sin(s1)]
- value: 0.4
- base: 0.59
- default: 0.0
- opt: SimulationParameters.OptParameterRange{Float64}(0.4, 0.0

[ Info: Merging temporary study files into "/Users/tims/Dropbox/FusionPowerPlantDesign_FUSE/FuseExamples/test_moop212123223/database.h5"...
[ Info: All files in `/Users/tims/Dropbox/FusionPowerPlantDesign_FUSE/FuseExamples/test_moop212123223/tmp_h5_output` are successfully merged into /Users/tims/Dropbox/FusionPowerPlantDesign_FUSE/FuseExamples/test_moop212123223/database.h5
[ Info: Cleaning up merged files from disk...
[ Info: Base directory '/Users/tims/Dropbox/FusionPowerPlantDesign_FUSE/FuseExamples/test_moop212123223/tmp_h5_output' and all its subdirectories were empty and have been removed.
[ Info: Merging temporary study files into "/Users/tims/Dropbox/FusionPowerPlantDesign_FUSE/FuseExamples/test_moop212123223/database.h5"...
[ Info: All files in `/Users/tims/Dropbox/FusionPowerPlantDesign_FUSE/FuseExamples/test_moop212123223/tmp_h5_output` are successfully merged into /Users/tims/Dropbox/FusionPowerPlantDesign_FUSE/FuseExamples/test_moop212123223/database.h5
[ Info: Cleaning u